# Posttest KB 6: Deep Neural Network (DNN)

* Import Library yang digunakan

In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense
from sklearn.impute import SimpleImputer

In [29]:
data = pd.read_csv('vehicle.csv')
data

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner
...,...,...,...,...,...,...,...,...
4335,Hyundai i20 Magna 1.4 CRDi (Diesel),2014,409999,80000,Diesel,Individual,Manual,Second Owner
4336,Hyundai i20 Magna 1.4 CRDi,2014,409999,80000,Diesel,Individual,Manual,Second Owner
4337,Maruti 800 AC BSIII,2009,110000,83000,Petrol,Individual,Manual,Second Owner
4338,Hyundai Creta 1.6 CRDi SX Option,2016,865000,90000,Diesel,Individual,Manual,First Owner


In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4340 entries, 0 to 4339
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           4340 non-null   object
 1   year           4340 non-null   int64 
 2   selling_price  4340 non-null   int64 
 3   km_driven      4340 non-null   int64 
 4   fuel           4340 non-null   object
 5   seller_type    4340 non-null   object
 6   transmission   4340 non-null   object
 7   owner          4340 non-null   object
dtypes: int64(3), object(5)
memory usage: 271.4+ KB


# Preprocessing dataset

* Melihat apakah ada data null atau duplikat

In [31]:
data.isna().sum()

name             0
year             0
selling_price    0
km_driven        0
fuel             0
seller_type      0
transmission     0
owner            0
dtype: int64

In [32]:
# data = pd.read_csv('Forbes Top2000 2017.csv')

# print(data.columns)

# print(data['Sector'].dtype)
# print(data['Industry'].dtype)

# simpleImputerObjek = SimpleImputer(strategy='most_frequent')
# data.loc[:, 'Sector'] = simpleImputerObjek.fit_transform(data[['Sector']])
# data.loc[:, 'Industry'] = simpleImputerObjek.fit_transform(data[['Industry']])

# print(data.isnull().sum())


* Mengubah Nilai Target Dengan Encoding

In [33]:
X = data.drop('selling_price', axis=1) 
y = data['selling_price']

In [34]:
label = LabelEncoder()
y = label.fit_transform(y)

In [35]:
# Memilih atribut yang dianggap memengaruhi hasil prediksi
selected_features = ['year', 'km_driven']
X = data[selected_features]
y = data['selling_price']

* Menerapkan One-Hot Encoding menggunakan get_dummies dari Pandas

In [36]:
try:
    X = pd.get_dummies(X, columns=['name', 'fuel', 'seller_type', 'transmission', 'owner'], drop_first=True)
except KeyError as e:
    print("KeyError:", e)
    print("Error Details:", str(e))

KeyError: "None of [Index(['name', 'fuel', 'seller_type', 'transmission', 'owner'], dtype='object')] are in the [columns]"
Error Details: "None of [Index(['name', 'fuel', 'seller_type', 'transmission', 'owner'], dtype='object')] are in the [columns]"


* Standarisasi

In [38]:
print('Nilai standarisasi sebelum dilakukan scaling: ')
std = data.std(numeric_only=True)
np.std(data)

Nilai standarisasi sebelum dilakukan scaling: 


year                  4.214858
selling_price    578482.079219
km_driven         46638.728140
dtype: float64

In [39]:
standard_scaler = StandardScaler()
x_standard = standard_scaler.fit_transform(
    data[['year', 'km_driven', 'selling_price']]
)

In [40]:
print('Nilai sesudah dilakukan scalling: ')
np.std(x_standard)

Nilai sesudah dilakukan scalling: 


1.0

# Split dataset menjadi train dan test

In [41]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state=42)

# Model sequential

In [59]:
# model = Sequential([
#     Dense(units=64, activation='relu',input_shape=(None,1)), #Output Layer Wajib units
#     Dense(units=32, activation='relu'),
#     Dense(units=1, activation='sigmoid')
# ])

# model.summary()

# model = Sequential(
#   [
#     layers.Dense(128, activation='relu', input_shape=(None, 1)),
#     layers.Dense(32, activation='relu'),
#     layers.Dense(3, activation='softmax')
#   ]
# )

model = Sequential([
    layers.Dense(128, activation='relu', input_shape=(None, 1)),
    layers.Dense(32, activation='relu'),
    layers.Dense(2, activation='sigmoid')  # Two units for binary classification
])


model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, None, 128)         256       
                                                                 
 dense_16 (Dense)            (None, None, 32)          4128      
                                                                 
 dense_17 (Dense)            (None, None, 2)           66        
                                                                 
Total params: 4450 (17.38 KB)
Trainable params: 4450 (17.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Model Training

__Model Compile__

In [60]:
# model.compile(
#     optimizer = 'adam',
#     loss = 'binary_crossentropy',
#     metrics=['accuracy']
# )

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

__Call Back__

* Callback untuk menghentikan pelatihan jika tidak ada perbaikan pada validation loss

In [61]:
stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

* Callback untuk menyimpan model terbaik berdasarkan validation loss

In [62]:
check = ModelCheckpoint('best_model.h5', save_best_only=True)

__Model Fit__

In [64]:
model.fit(
    x = x_train,
    y = y_train,
    epochs=100, #100x Pembelajaran
    validation_data = (X, y),
    batch_size=64,
    callbacks=[stop, check]
)

# model.fit(x_train, y_train, batch_size=5, epochs=50)



Epoch 1/100


ValueError: in user code:

    File "C:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1127, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1185, in compute_loss
        return self.compiled_loss(
    File "